In [17]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_integer_dtype
# import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt


In [9]:
dt = pd.read_csv('./data_test/protection_NHDPv2_WBD.csv')
dt.head()

c:\users\goura\.virtualenvs\ml\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,HUC_12,PGAPSTAT3,PIUCN_IA,PIUCN_II,PIUCN_III,PIUCN_IV,PIUCN_V,PIUCN_VI,PIUCN_IB,PIUCN_ALL,PGAPSTAT1_2,PGAPSTAT123
0,10100020101,40.4681,0.3125,0,0,0,0,0,0.105501,0.418001,0.970601,41
1,10100020102,49.7524,1.24451e-08,0,0,0,0,0,0.0288002,0.0288002,0.0288002,50
2,10100020103,70.0271,1.55307e-08,0,0,0,0,0,0.6305,0.6305,19.146,89
3,10100020104,45.9408,0,0,0,0,0,0,1.0092,1.0092,1.0092,47
4,10100020105,55.4016,8.54192e-09,0,0,0,0,0,18.6537,18.6537,18.6537,74


In [21]:
t1 = dt['PIUCN_IB']
t2 = dt['PIUCN_ALL']
print("{0} , {1}".format(len(t1), len(t2)))
# st.entropy(t1, t2)
# p = np.asarray(t1, dtype=np.float)
p = t1
q = np.asarray(t2, dtype=np.float)
# q = t2
np.sum(np.where(p != 0, p * np.log(p / q), 0))

82915 , 82915


ValueError: could not convert string to float: 'None'

In [3]:
# print(dt.dtypes)
# dt['PGAPSTAT3'].apply(lambda x: x.strip())
print(is_numeric_dtype(dt['PGAPSTAT123']))
t = pd.to_numeric(dt['PGAPSTAT123'], errors='coerce')
t = t.dropna()
t.isnull().sum()

False


0

In [4]:
l1 = [1,2,3.0,1]
l2 = [1,2,3.0,2]
l3 = [1,2,"3A","BLAH"]
d = []
d.append(l1); d.append(l2); d.append(l3); 
df = pd.DataFrame(d, columns=("a","b","c","d"))
print(df)
print(df.dtypes)
is_numeric_dtype(df["c"])

def clean_objects_if_numeric(s:pd.Series):
    t = pd.to_numeric(s, errors='coerce')
    t_reduced = t.dropna()
    return t_reduced
clean_objects_if_numeric(df["d"])
df["c"]

   a  b   c     d
0  1  2   3     1
1  1  2   3     2
2  1  2  3A  BLAH
a     int64
b     int64
c    object
d    object
dtype: object


0     3
1     3
2    3A
Name: c, dtype: object

In [5]:
# fake1 = [1990 for i in range(0, 40)]
fake1 = np.linspace(0, 40, 40, dtype=int)
# fake1
# replace_at = 
# np.random.random_integers(1,5)
print(fake1)
replace_at = np.random.randint(0, 40, size=10)
for i in range(0, len(fake1)):
    if(i in replace_at):
        fake1[i] = np.random.randint(0, 40)
print(fake1)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40]
[30  1  2  3  4  5 14  7  8  9 10 11 12 13 13 15 16 17 18  8 20 21 22 23 25
 25 26 27 28 31 30 31 32 20 34 35 16 37 38 40]


In [6]:
col1 = dt['Year'].values
col2 = dt['Total'].values
col3 = dt['Foreign'].values
col4 = dt['Domestic'].values
col4.ravel()

KeyError: 'Year'

In [ ]:
# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [st.uniform, st.norm, st.zipf, st.linregress]
#     DISTRIBUTIONS = [st.linregress]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data) #TODO Study return parameters for different distributions

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

In [ ]:
# Load data from statsmodels datasets
# data = pd.Series(sm.datasets.elnino.load_pandas().data.set_index('YEAR').values.ravel())
# data = pd.Series(fake1)
data = pd.Series(col1)

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, normed=True, alpha=0.5, color=plt.rcParams['axes.color_cycle'][1])
# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_name, best_fir_paramms = best_fit_distribution(data, 200, ax)
best_dist = getattr(st, best_fit_name)
print("{0} , {1}, {2}".format(best_fit_name, best_fir_paramms, best_dist ))

In [ ]:
best_dist